In [1]:
##Brute-Force, Exhaustive Search and Branch&Bound!

import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import scipy.stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression  
import itertools
import copy

data = pd.read_csv("/Users/geoffreykasenbacher/Desktop/projects/branch and bound/Branch-and-Bound/dataset.csv",sep=";").astype(float)

In [38]:
## Get Data as a Pandas Dataframe! Make sure to have numerical data only! (pd.get_dummies might help)

file ="/Users/geoffreykasenbacher/Desktop/projects/branch and bound/Branch-and-Bound/dataset.csv"

data = pd.read_csv(file,sep=";").astype(float)

Y = list(data)[0]
X = list(data)[1:]



In [39]:
## exhaustive search - Brute Force. create class here

class BruteForce: 
    
    def __init__(self, Y, X, data):
        self.Y = Y
        self.X = X
        self.data = data
   
    def powerset(self, X):
        target = []
        def combinations(target,X):
            for i in range(len(X)):
                new_target = copy.copy(target)
                new_data = copy.copy(X)
                new_target.append(X[i])
                new_data = X[i+1:]
                pp.append(new_target)
                #print(new_target)
                combinations(new_target,new_data)
        pp = []
        combinations(target, X)
        return pp
    
    def regress(self, X, Y, data):
        regressor = LinearRegression()
        R = regressor.fit(data[X],data[Y])  
        s = regressor.score(data[X], data[Y])
        return (s) 
    
    def search(self):
        solutions = []
        combos = self.powerset(self.X)
        for i in combos:
            solutions.append(self.regress(i, self.Y, self.data))  
        return solutions, combos
    
    def solve(self):
        sol, combos = self.search()
        highest_R2 = combos[np.argmax(sol)]
        R2 = max(sol)
        
        print("higehst R^2 is", R2, "with dependant variables:", highest_R2)
        return highest_R2
        
                 

In [43]:
t = BruteForce(Y,X,data).solve()


higehst R^2 is 0.032779569202025516 with dependant variables: ['SEX', 'INCOME', 'EDUCATION', 'AGE', 'LOCATION']


In [64]:
## this will solve all possible linear regressions and return the one with highest R^2

class exhaustivesearch: 
    
    def __init__(self, data):
        self.data = data
        
    def assign(self):
        XX = list(self.data)
        dic = {}
        
        for i in XX:
            Y = i
            X = self.powerset([x for x in XX if x != Y])
            dic[Y] = X
        return dic

    def powerset(self, X):
        target = []
        def combinations(target,X):
            for i in range(len(X)):
                new_target = copy.copy(target)
                new_data = copy.copy(X)
                new_target.append(X[i])
                new_data = X[i+1:]
                pp.append(new_target)
                combinations(new_target,new_data)
        pp = []
        combinations(target, X)
        return pp
    
    def regress(self, X, Y, data):
        regressor = LinearRegression()
        R = regressor.fit(self.data[X],self.data[Y])  
        s = regressor.score(self.data[X], self.data[Y])
        return (s) 
    
    def search(self):
        dic = self.assign()
        newdic = {}
        for y in dic:
            for x in dic[y]:
                newdic[self.regress(x, y, data)] = y, x
        new={}
        sd = sorted(newdic.items(), reverse=True)
        for k,v in sd:
            new[k] = v
        return new

    def solve(self):
        results = self.search()
        highest_R2 = list(results.keys())[0]
        best_vars = results[highest_R2]
        
        print("higehst R^2 is", highest_R2, "with dependant/indepedant variables:", best_vars)
        return highest_R2

In [65]:
wow = exhaustivesearch(data).solve()

higehst R^2 is 0.2968502113618645 with dependant/indepedant variables: ('INCOME', ['VIOLENCE', 'SEX', 'EDUCATION', 'AGE', 'LOCATION'])


In [68]:
f = "/Users/geoffreykasenbacher/Downloads/bank/bank.csv"
test = pd.read_csv(f,sep=";") #.astype(float)
test1 = pd.get_dummies(test)

test2 = test1[list(test1)[:3]]

Y = list(data)[0]
X = list(data)[1:]


In [69]:
so = exhaustivesearch(test2).solve()

higehst R^2 is 0.007319114927034076 with dependant/indepedant variables: ('age', ['balance', 'day'])
